# Chapter 13 - Loading and Preprocessing Data with Tensorflow

## Exercises

1. Why would you want to use the Data API?
    sebaiknya menggunakan Data API (tf.data) untuk membangun pipeline input yang sangat efisien dan cepat. Ini memungkinkan pemrosesan data dalam jumlah besar secara paralel, memuat data di latar belakang (prefetching) untuk menjaga agar GPU tetap sibuk, dan menyederhanakan manipulasi dataset yang kompleks.

2. What are the benefits of splitting a large dataset into multiple files?
    Membagi dataset besar menjadi beberapa file memungkinkan shuffling yang lebih baik dengan mencampur urutan file dan isinya. Ini juga memfasilitasi pemuatan dan pemrosesan data secara paralel dari beberapa file sekaligus, yang secara signifikan mempercepat training.

3. During training, how can you tell that your input pipeline is the bottleneck? What can you do to fix it?
    dapat mengetahui pipeline input menjadi bottleneck jika utilisasi GPU rendah selama training. Untuk memperbaikinya, tambahkan .prefetch(tf.data.AUTOTUNE) di akhir pipeline, paralelkan transformasi data menggunakan .map() dengan num_parallel_calls, dan gunakan .cache() jika dataset muat di memori.

4. Can you save any binary data to a TFRecord file, or only serialized protocol buffers?
    Ya, dapat menyimpan data biner apa pun dalam file TFRecord. Format ini pada dasarnya adalah wadah untuk urutan catatan biner, yang bisa berupa serialized protocol buffer, gambar JPEG, atau data biner lainnya.

5. Why would you go through the hassle of converting all your data to the Example protobuf format? Why not use your own protobuf definition?
    Menggunakan format tf.train.Example yang standar memastikan kompatibilitas dengan ekosistem TensorFlow yang lebih luas, seperti TF Transform dan TFX. Ini menghilangkan kebutuhan untuk menulis dan mengelola definisi sendiri dan sudah cukup fleksibel untuk sebagian besar kasus penggunaan.

6. When using TFRecords, when would you want to activate compression? Why not do it systematically?
    Aktifkan kompresi pada TFRecords untuk menghemat ruang disk atau bandwidth jaringan, terutama saat membaca dari penyimpanan jarak jauh. Namun, jangan melakukannya secara sistematis karena proses dekompresi memerlukan waktu CPU tambahan, yang dapat memperlambat pipeline data jika sudah terikat oleh CPU (CPU-bound).

7. Data can be preprocessed directly when writing the data files, or within the tf.data pipeline, or in preprocessing layers within your model, or using TF Transform. Can you list a few pros and cons of each option?
    Pra-pemrosesan saat menulis file sangat cepat saat training tetapi tidak fleksibel. Pra-pemrosesan dalam pipeline tf.data fleksibel tetapi dapat memperlambat training. Pra-pemrosesan di dalam lapisan model menyederhanakan deployment tetapi menambah beban saat inferensi. TF Transform adalah yang paling kuat karena menggabungkan analisis dataset penuh dengan eksekusi yang efisien, mencegah training-serving skew, tetapi menambah kompleksitas pada alur kerja Anda.

8. Name a few common techniques you can use to encode categorical features. What about text?
    Untuk fitur kategorikal, teknik umum adalah One-Hot Encoding atau Embeddings, di mana embeddings lebih disukai dalam deep learning karena menangkap hubungan antar kategori. Untuk teks, pendekatan berkisar dari Bag-of-Words dan TF-IDF yang sederhana hingga Word Embeddings dan model Transformer canggih seperti BERT yang menangkap konteks.

In [2]:
import tensorflow as tf

In [3]:
X = tf.range(10)

In [4]:
dataset = tf.data.Dataset.from_tensor_slices(X)

In [5]:
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [6]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [7]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [8]:
dataset = dataset.map(lambda x: x * 2) # Items: [0,2,4,6,8,10,12]

In [9]:
dataset = dataset.apply(tf.data.experimental.unbatch()) # Items: 0,2,4,...

Instructions for updating:
Use `tf.data.Dataset.unbatch()`.


In [10]:
dataset = dataset.filter(lambda x: x < 10) # Items: 0 2 4 6 8 0 2 4 6...

In [11]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)


In [13]:
dataset = tf.data.Dataset.range(10).repeat(3) # 0 to 9, three times
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


In [ ]:
train_filepaths

In [ ]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

In [ ]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)